Model progress can be recorded during and after training. This means that a model can pick up where it left off and avoid long periods of training. We should know how to save complex machine learning models so that we can share our model and our team members can recreate our work.

We should know how to save machine learning models because in the professional world we will be working with as complex machine learning models which will need us to work within teams and will require days. So to share our work within the teams and continue where we left will always require us to know how we can save machine learning models.

To demonstrate how to save machine learning models, we will first train a TensorFlow model because these models are large and complex enough that they need to be saved and worked within teams. 

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

C:\Users\Waqas.Ali\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Waqas.Ali\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Waqas.Ali\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Waqas.Ali\AppData\Roaming\Pytho

Now, we need a dataset to train our model. We will use the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# Building a Model
We will build a simple sequential model:

In [21]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])

    return model

In [22]:
# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### To Save Checkpoints While Training
We can use a trained model without having to retrain it or resume training where we left off, in case the training process is interrupted. The ModelCheckpoint callback method allows us to permanently save the model during and at the end of the training:

In [23]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [24]:
# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 928/1000 [==========================>...] - ETA: 0s - loss: 4.1631 - sparse_categorical_accuracy: 0.4095
Epoch 00001: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
1000/1000 [==============================] - 4s 4ms/sample - loss: 4.0291 - sparse_categorical_accuracy: 0.3970 - val_loss: 2.2972 - val_sparse_categorical_accuracy: 0.3190
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 2.2978 - sparse_categorical_accuracy: 0.2323
Epoch 00002: saving model to training_1/cp.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
1000/1000 [==============================] - 1s 1ms/sample - loss: 2.2980 - sparse_categorical_accuracy: 0.2310 - val_loss: 2.3026 - val_sparse_categorical_accuracy: 0.1950
Epoch 3/10
 896/1000 [===========

Create a new untrained model. When we are restoring a model from weights only, we must have another model with the same architecture as the trained model. Because this is the same model architecture, we can share weights even if it is a different instance of the model.

Now rebuild a new untrained model and rate it on the test set. An untrained model will run at random levels (about 10% accuracy):

In [25]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

 - 0s - loss: 9.0434 - sparse_categorical_accuracy: 0.1080
Untrained model, accuracy: 10.80%


Now load the weights and reevaluate:

In [26]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

 - 0s - loss: 2.3026 - sparse_categorical_accuracy: 0.1840
Restored model, accuracy: 18.40%


### Callback Checkpoint
Train a new model with unique checkpoints:

In [27]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00005: saving model to training_2/cp-0005.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00010: saving model to training_2/cp-0010.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00015: saving model to training_2/cp-0015.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00020: saving model to training_2/cp-0020.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00025: saving model to training_2/cp-0025.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00030: saving model to training_2/cp-0030.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00035: saving model to training_2/cp-0035.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00040: saving model to training_2/cp-0040.ckpt

Consider using a TensorFlow optimizer from `tf.train`.

Epoch 00045: saving model to training_2/cp-0045.ckpt

Consider 

Now, reset the model and load the latest checkpoint:

In [29]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [30]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

 - 0s - loss: 2.1368 - sparse_categorical_accuracy: 0.4310
Restored model, accuracy: 43.10%


### Manually save weights

Now, let’s see how we can manually save weights of the model:

In [31]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))


Consider using a TensorFlow optimizer from `tf.train`.
 - 0s - loss: 2.1368 - sparse_categorical_accuracy: 0.4310
Restored model, accuracy: 43.10%


### To Save Entire Machine Learning Model

`Call model.save` to save the architecture, weights, and training configuration of a model in a single file/folder. This will allow us to export a model so that it can be used without having the code that is used in training of the model. Since the optimizer state is recovered, we can resume training exactly where we left off.

In [32]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 3.6153 - sparse_categorical_accuracy: 0.2450
Epoch 2/5
1000/1000 [==============================] - 1s 746us/sample - loss: 2.3026 - sparse_categorical_accuracy: 0.2110
Epoch 3/5
1000/1000 [==============================] - 1s 695us/sample - loss: 2.3003 - sparse_categorical_accuracy: 0.2120
Epoch 4/5
1000/1000 [==============================] - 1s 730us/sample - loss: 2.3003 - sparse_categorical_accuracy: 0.2010
Epoch 5/5
1000/1000 [==============================] - 1s 582us/sample - loss: 2.3026 - sparse_categorical_accuracy: 0.2070


This will save our model in the directory.